In [1]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

In [2]:
import urllib.parse
import urllib.request
import json
import pandas as pd

estat-APIからレスポンスを取得する関数

In [3]:
def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [17]:
params = {
    'statsDataId':'0003130803',
}

In [19]:
res = get_estat_response(params)

In [42]:
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
print(CLASS_OBJ)

def changes(id,code):
  print(id)
  test = [d.get('CLASS') for d in CLASS_OBJ]
  test2 = [d.get('@code') for d in test]
  print(test)
  print(test2)


[{'@id': 'tab', '@name': '表章項目', 'CLASS': {'@code': '120', '@name': 'トン数', '@level': '', '@unit': 'トン'}}, {'@id': 'cat01', '@name': '輸移出入', 'CLASS': [{'@code': '100', '@name': '合計', '@level': '1'}, {'@code': '110', '@name': '輸出', '@level': '2', '@parentCode': '100'}, {'@code': '120', '@name': '輸入', '@level': '2', '@parentCode': '100'}, {'@code': '130', '@name': '移出', '@level': '2', '@parentCode': '100'}, {'@code': '140', '@name': '移入', '@level': '2', '@parentCode': '100'}]}, {'@id': 'cat02', '@name': 'T/S', '@description': '「Ｔ／Ｓ」とはトランシップをいい、「一般」の内数として別掲した。', 'CLASS': [{'@code': '100', '@name': '計', '@level': '1'}, {'@code': '110', '@name': '内T/S', '@level': '2', '@parentCode': '100'}]}, {'@id': 'cat03', '@name': '輸送形態', '@description': '「一般」 には、下記以外の貨物を計上した。\r\n「自動車航送船」には、自動車航送船により輸送されたバス、トラック、乗用車等（二輪自動車、自転車及び商品としての自動車を除く。）を計上した。\r\n「鉄道連絡船」 には、鉄道連絡船により輸送された貨物（自動車等を含む。）を計上した。', 'CLASS': [{'@code': '100', '@name': '計', '@level': '1'}, {'@code': '110', '@name': '一般', '@level': '2', '@pare

In [ ]:
changes('tab','120')

In [15]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  print(CLASS_OBJ)

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  for row in df_res.itertuples():
    # row['@tab'] = row['@tab']
    row['@tab'] = 'test'

  # # CLASS_OBJのDataFrameを結合
  # for d in CLASS_OBJ :
  #   # DataFrameに変換 '@code','@name'だけ抽出
  #   df_class = pd.json_normalize(d['CLASS']) 
  #   df_class = df_class.copy()[['@code','@name']]

  #   # @codeをキー名に変更 @nameをキー名＋'_name'に変更
  #   key_name = '@{}'.format(d['@id'])
  #   df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

  #   # DataFrameを結合
  #   df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # # 統計情報を追加
  # df_res['statsDataId'] = TABLE_INF['@id']
  # df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

In [ ]:
params = {
    'statsDataId':'0003130803',
}

res = get_estat_response(params)

